In [7]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession

import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, mean_absolute_error, mean_squared_error, r2_score


In [ ]:
my_LR_data = np.loadtxt("../coursera_ex_2/ex2data1.txt", delimiter=",")
X = np.c_[np.ones(my_LR_data.shape[0]), my_LR_data[:, [0,1]]]
y = np.c_[my_LR_data[:, [2]]]

Convert the data into test and training set

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, random_state=42)